In [105]:
import pandas as pd
import glob
import numpy as np

In [106]:
#ファイル読み込み
df = pd.DataFrame()
for i in glob.glob("train/*.csv*"):
    tmp_df=pd.read_csv(i)
    df = pd.concat([df, tmp_df],ignore_index=True)

C:\Users\osako\AppData\Local\Temp\ipykernel_4668\132664144.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  tmp_df=pd.read_csv(i)
C:\Users\osako\AppData\Local\Temp\ipykernel_4668\132664144.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  tmp_df=pd.read_csv(i)
C:\Users\osako\AppData\Local\Temp\ipykernel_4668\132664144.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  tmp_df=pd.read_csv(i)
C:\Users\osako\AppData\Local\Temp\ipykernel_4668\132664144.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  tmp_df=pd.read_csv(i)


In [107]:
#欠損値確認,欠損値削除
df_rownum=df.shape[0]
df_null_column=df.columns[df.isnull().sum()==df_rownum]
df=df.drop(df_null_column,axis=1)

In [108]:
df.drop(['ID',
         '種類',
#                '地区名',
#                '最寄駅：名称',
        '市区町村コード',
#                '用途',
#                '市区町村名',
#                '間取り',
#                '建物の構造',
#                '都市計画'
              ],inplace=True,axis=1)
df["取引の事情等"].fillna(0, inplace=True)
df.loc[df["取引の事情等"] != 0, "取引の事情等"] = 1

In [109]:
#欠損値の多い列を削除
df=df.drop(['今後の利用目的'],axis=1)

In [110]:
#各列のユニークな値を抽出
unique_value=df.apply(pd.Series.unique)
# print(unique_value['都道府県名'])

In [111]:
df.sort_values('取引時点', inplace=True)
df.reset_index(drop=True, inplace=True)

In [112]:
df_edit1=df.copy()
df_edit1.dropna(inplace=True)
df_edit1.reset_index(drop=True, inplace=True)

In [113]:
val_min_idx = min(df_edit1[df_edit1['取引時点'].str.contains('2021年第4四半期|2022年第1四半期', regex=True)].index)
# test_min_idx = min(df[df['取引時点'].str.contains('2022年第2四半期|2022年第3四半期', regex=True)].index)
val_min_idx
# test_min_idx

547549

In [114]:
enc_dic = {}
for i, e in enumerate(sorted(list(set(df['取引時点'].values)))):
    enc_dic[e] = i
df['取引時点_enc'] = df['取引時点'].map(enc_dic)

In [117]:
TARGET = '取引価格（総額）_log'
te_dic = {}
time_col = '取引時点_enc'
group_col = '都道府県名'

for i in set(df[time_col].values):
    #'取引時点_encがiより小さい行だけ取り出されている
    #各年の四半期までの平均ではなく、その時点までの平均を計算しています。
    tmp_df = df[df[time_col] < i]
    te_dic[i] = tmp_df.groupby(group_col)[TARGET].agg('mean').to_dict()

te_dic

{0: {},
 1: {'京都府': 7.113646169814669,
  '埼玉県': 7.201547058894921,
  '大阪府': 7.115043783394055,
  '岐阜県': 7.143333518213244,
  '愛知県': 7.091347948095394,
  '東京都': 7.287061204746857,
  '神奈川県': 7.200666282181238},
 2: {'京都府': 7.136029734002714,
  '兵庫県': 6.738560627359831,
  '埼玉県': 7.198055205022039,
  '大阪府': 7.125645900748962,
  '岐阜県': 7.014493340289938,
  '愛知県': 7.086638676984786,
  '東京都': 7.300552712053001,
  '神奈川県': 7.206827694489212},
 3: {'京都府': 7.15199002373719,
  '兵庫県': 6.982555895886166,
  '北海道': 7.332532791770711,
  '千葉県': 6.943422600311213,
  '埼玉県': 7.261219805020934,
  '大阪府': 7.139638350220338,
  '宮城県': 7.258997194952134,
  '岐阜県': 6.871932255580502,
  '愛知県': 7.098305068525317,
  '東京都': 7.31996334938226,
  '神奈川県': 7.210521959869596,
  '福岡県': 6.9956130378462476},
 4: {'京都府': 7.143623421559968,
  '兵庫県': 7.046313166470019,
  '北海道': 6.94309795648886,
  '千葉県': 7.111356971407157,
  '埼玉県': 7.239019773961096,
  '大阪府': 7.138163859913052,
  '宮城県': 6.981459029420642,
  '岐阜県': 6.8914366711497

In [116]:
df

,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log,取引時点_enc
0,東京都,板橋区,中台,志村三丁目,12,３ＬＤＫ,70,昭和53年,ＳＲＣ,住宅,第１種中高層住居専用地域,60.0,200.0,2005年第3四半期,改装済,0,7.342423,0
1,京都府,京都市上京区,出水学区,二条,NaN,オープンフロア,35,昭和50年,ＲＣ,店舗,商業地域,80.0,600.0,2005年第3四半期,未改装,0,7.000000,0
2,東京都,板橋区,蓮根,西台,NaN,３ＬＤＫ,75,平成17年,ＲＣ,住宅,第１種住居地域,60.0,300.0,2005年第3四半期,NaN,0,7.602060,0
3,東京都,豊島区,池袋本町,NaN,NaN,１Ｋ,25,NaN,NaN,住宅,NaN,NaN,NaN,2005年第3四半期,未改装,0,7.041393,0
4,神奈川県,横浜市保土ケ谷区,新桜ケ丘,東戸塚,30分?60分,２ＤＫ,45,昭和53年,NaN,住宅,NaN,NaN,NaN,2005年第3四半期,改装済,0,6.857332,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765407,大阪府,大阪市港区,波除,弁天町,6,NaN,85,平成21年,ＲＣ,住宅,準工業地域,60.0,300.0,2022年第1四半期,NaN,0,7.602060,66
765408,山口県,宇部市,若松町,宇部新川,3,４ＬＤＫ,95,平成18年,ＲＣ,住宅,近隣商業地域,80.0,300.0,2022年第1四半期,NaN,0,7.342423,66
765409,静岡県,沼津市,杉崎町,沼津,13,２ＬＤＫ,55,昭和57年,ＲＣ,住宅,第２種住居地域,60.0,200.0,2022年第1四半期,改装済,0,6.477121,66
765410,大阪府,大阪市淀川区,十三東,十三,5,１Ｋ,25,平成21年,ＲＣ,住宅,商業地域,80.0,400.0,2022年第1四半期,未改装,0,7.278754,66


In [103]:
TARGET = '取引価格（総額）_log'
time_col = '取引時点_enc'
group_col = '都道府県名'
def calc_te(row):
    #辞書の中のdfの'取引時点_enc'かつ四半期の都道府県
    if row[time_col] in te_dic and row[group_col] in te_dic[row[time_col]]:
        return te_dic[row[time_col]][row[group_col]]
    else:
        return 0

df[group_col+'_te'] = df.apply(calc_te, axis=1)
df.head(2)

,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log,取引時点_enc,都道府県名_te
0,東京都,板橋区,中台,志村三丁目,12,３ＬＤＫ,70,昭和53年,ＳＲＣ,住宅,第１種中高層住居専用地域,60.0,200.0,2005年第3四半期,改装済,0,7.342423,0,0.0
1,京都府,京都市上京区,出水学区,二条,NaN,オープンフロア,35,昭和50年,ＲＣ,店舗,商業地域,80.0,600.0,2005年第3四半期,未改装,0,7.000000,0,0.0


In [64]:
df_edit1.loc[df_edit1['建築年']=='戦前','建築年']='昭和20年'
df_edit1['和暦名称']=df_edit1['建築年'].str[:2]
df_edit1['西暦']=df_edit1['建築年'].str[2:-1]
df_edit1['西暦'].unique()
df_edit1['西暦']=df_edit1['西暦'].astype(int)
print(df_edit1['西暦'].unique())
df_edit1.loc[df_edit1['和暦名称']=='昭和','西暦']+=1925
df_edit1.loc[df_edit1['和暦名称']=='平成','西暦']+=1988
df_edit1.loc[df_edit1['和暦名称']=='令和','西暦']+=2018
df_edit1.drop(['和暦名称','建築年'],inplace=True,axis=1)
df_edit1.loc[df_edit1['最寄駅：距離（分）']=='30分?60分','最寄駅：距離（分）']=45
df_edit1.loc[df_edit1['最寄駅：距離（分）']=='1H?1H30','最寄駅：距離（分）']=75
df_edit1.loc[df_edit1['最寄駅：距離（分）']=='1H30?2H','最寄駅：距離（分）']=105
df_edit1.loc[df_edit1['最寄駅：距離（分）']=='2H?','最寄駅：距離（分）']=120
df_edit1['取引時点年数']=df_edit1['取引時点'].str[:4]
df_edit1['四半期']=df_edit1['取引時点'].str[5:]
df_edit1.loc[df_edit1['四半期']=='第2四半期','取引時点年数']+='.25'
df_edit1.loc[df_edit1['四半期']=='第3四半期','取引時点年数']+='.5'
df_edit1.loc[df_edit1['四半期']=='第4四半期','取引時点年数']+='.75'
df_edit1.drop(['取引時点'],
#                '四半期'],
              inplace=True,axis=1)

[53 17 64 10 59  3 47 57 60 50 11 61  6 63 13  5  7 55 12  2 20 49 58 62
 54 14 52 56 15 45 42  9  4 43 48  8 16 41 51 46 44 36 39 40 38 18 31 28
 35 33 19 34 32 27 21 22 37 24 23 25 29 26 30]


In [65]:
df_edit1.loc[df_edit1['面積（㎡）']=='2000㎡以上','面積（㎡）']='2000'

In [66]:
df_edit1['最寄駅：距離（分）']=df_edit1['最寄駅：距離（分）'].astype(float)
df_edit1['面積（㎡）']=df_edit1['面積（㎡）'].astype(int)
df_edit1['西暦']=df_edit1['西暦'].astype(int)
df_edit1['取引時点年数']=df_edit1['取引時点年数'].astype(float)
df_edit1['容積率（％）']=df_edit1['容積率（％）'].astype(float)
df_edit1['建ぺい率（％）']=df_edit1['建ぺい率（％）'].astype(float)
df_edit1['取引価格（総額）_log']=df_edit1['取引価格（総額）_log'].astype(float)

In [67]:
df_test=df_edit1['取引価格（総額）_log']
df_edit1.drop('取引価格（総額）_log',inplace=True,axis=1)

In [68]:
# df_dummy=pd.get_dummies(df_edit1[[
# #     '市区町村コード',
#     '都道府県名',
#     '市区町村名',
#     '最寄駅：名称',
#      '間取り',
#      '建物の構造',
# #     '用途',
#     '都市計画',
#     '改装']],drop_first=True)
# df_dummy = pd.concat([df_edit1, df_dummy], axis=1)
# df_dummy.drop(['都道府県名', 
#                '市区町村名',
#                '間取り',
#                 '建物の構造',
#                 '都市計画',
#                '改装'],axis=1,inplace=True)

In [69]:
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder()

labels = df_edit1[[
#     '市区町村コード',
    '都道府県名',
    '市区町村名',
    '最寄駅：名称',
     '間取り',
     '建物の構造',
    '地区名',
     '用途',
    '都市計画',
    '四半期',
    '改装']].values
labels_id = oe.fit_transform(labels)
# df_dummy = pd.concat([df_edit1, labels_id], axis=1)
# df_dummy.drop(['都道府県名', 
#                '市区町村名',
#                '間取り',
#                 '建物の構造',
#                 '都市計画',
#                '改装'],axis=1,inplace=True)
df_dummy = pd.concat([df_edit1.reset_index(drop=True), pd.DataFrame(labels_id)], axis=1)
df_dummy.drop(['都道府県名',
               '市区町村名', 
               '四半期',
               '間取り', 
               '建物の構造', 
               '都市計画',
               '最寄駅：名称',
               '地区名',
               '用途',
               '改装'], axis=1, inplace=True)
print(labels_id.shape) 

(564693, 10)


In [70]:
df_dummy

,最寄駅：距離（分）,面積（㎡）,建ぺい率（％）,容積率（％）,取引の事情等,西暦,取引時点年数,0,1,2,3,4,5,6,7,8,9
0,12.0,70,60.0,200.0,0,1978,2005.5,26.0,363.0,1698.0,38.0,7.0,847.0,6.0,7.0,2.0,0.0
1,11.0,20,80.0,600.0,0,2005,2005.5,26.0,314.0,1666.0,6.0,4.0,4830.0,6.0,0.0,2.0,1.0
2,8.0,20,80.0,600.0,0,1989,2005.5,26.0,551.0,720.0,15.0,7.0,2402.0,6.0,0.0,2.0,1.0
3,14.0,20,80.0,300.0,0,1998,2005.5,26.0,348.0,3059.0,6.0,4.0,2723.0,6.0,13.0,2.0,1.0
4,10.0,60,60.0,200.0,0,1984,2005.5,7.0,9.0,754.0,38.0,4.0,1633.0,6.0,10.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564688,18.0,85,60.0,200.0,0,2021,2022.0,9.0,153.0,722.0,48.0,4.0,11355.0,6.0,7.0,0.0,1.0
564689,25.0,25,60.0,200.0,0,1974,2022.0,42.0,414.0,3404.0,6.0,7.0,237.0,6.0,4.0,0.0,1.0
564690,2.0,25,80.0,300.0,0,2011,2022.0,26.0,35.0,3022.0,6.0,4.0,9693.0,6.0,13.0,0.0,1.0
564691,13.0,55,60.0,200.0,0,1982,2022.0,42.0,402.0,2417.0,26.0,4.0,6506.0,6.0,12.0,0.0,0.0


In [71]:
df_dummy['面積（㎡）']=np.log(df_dummy['面積（㎡）'])
df_dummy['面積（㎡）'].replace([np.inf, -np.inf], np.nan, inplace=True)
df_dummy['面積（㎡）'].fillna(0,inplace=True)

In [72]:
df_dummy['最寄駅：距離（分）']=np.log(df_dummy['最寄駅：距離（分）'])
df_dummy['最寄駅：距離（分）'].replace([np.inf, -np.inf], np.nan, inplace=True)
df_dummy['最寄駅：距離（分）'].fillna(0,inplace=True)

c:\users\osako\pycharmprojects\pythonproject2\venv\lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [73]:
df_dummy['建ぺい率×容積率']=df_dummy['建ぺい率（％）']*df_dummy['容積率（％）']

In [74]:
df_dummy['敷地面積']=df_dummy['面積（㎡）']/df_dummy['建ぺい率（％）']*100

In [75]:
df_dummy['延べ床面積']=df_dummy['容積率（％）']/100*df_dummy['敷地面積']

In [76]:
df_dummy['年差']=df_dummy['西暦']-df_dummy['取引時点年数']

In [77]:
df_dummy.isnull().sum()

最寄駅：距離（分）    0
面積（㎡）        0
建ぺい率（％）      0
容積率（％）       0
取引の事情等       0
西暦           0
取引時点年数       0
0            0
1            0
2            0
3            0
4            0
5            0
6            0
7            0
8            0
9            0
建ぺい率×容積率     0
敷地面積         0
延べ床面積        0
年差           0
dtype: int64

In [78]:
# df_dummy['最寄駅：距離（分）'].max()

In [79]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(df_dummy, df_test, test_size=0.2, random_state=41)

In [88]:
df_test

0         7.342423
1         7.301030
2         6.857332
3         7.176091
4         7.204120
            ...   
564688    7.740363
564689    4.698970
564690    7.447158
564691    6.477121
564692    7.278754
Name: 取引価格（総額）_log, Length: 564693, dtype: float64

In [92]:
X_train=df_dummy.iloc[:val_min_idx,:]
y_train=df_test.iloc[:val_min_idx]
X_test=df_dummy.iloc[val_min_idx:,:]
y_test=df_test.iloc[val_min_idx:]

In [93]:
y_train

0         7.342423
1         7.301030
2         6.857332
3         7.176091
4         7.204120
            ...   
547544    7.591065
547545    6.892095
547546    7.716003
547547    6.851258
547548    7.462398
Name: 取引価格（総額）_log, Length: 547549, dtype: float64

In [95]:
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge,Lasso
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
import numpy as np
xgb_r = xgb.XGBRegressor(objective='reg:linear',
                          n_estimators=500,
                          max_depth=8,
#                           gamma=5,
                          seed=113)

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor',xgb_r)
])
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_train)
# y_trueは実際の値、y_predは予測値
mse = mean_squared_error(y_train, y_pred)
rmse = np.sqrt(mse)
print("RMSE:", rmse)
y_pred = pipe.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae=mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)
print("RMSE:", rmse)
print(mae)

[22:18:00] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
RMSE: 0.09463072358196951
RMSE: 0.13112752551525989
0.08140852497861883


In [ ]:
# coefficients = pipe.named_steps['regressor'].coef_
# # 各特徴量の係数を出力
# for i, col_name in enumerate(X_train.columns):
#     print(f"{col_name}: {coefficients[i]}")

In [ ]:
pipe.named_steps

In [ ]:
df_dummy

In [ ]:
import seaborn as sns
import japanize_matplotlib
import matplotlib.pyplot as plt
# sns.set(font='Hiragino Sans')zz
sns.pairplot(df_dummy[['最寄駅：距離（分）', '面積（㎡）', '建ぺい率（％）','容積率（％）','西暦','取引時点年数',]])

In [ ]:
plt.scatter(x=(df_dummy['西暦']), y=np.log(df_dummy['面積（㎡）']))

In [ ]:
df_dummy['最寄駅：距離（分）']

In [ ]:
np.log(df_dummy['最寄駅：距離（分）']).isnull().sum()

In [ ]:
import japanize_matplotlib
import matplotlib.pyplot as plt
plt.hist(df_dummy['西暦']-df_dummy['取引時点年数'])